Importando Libs

In [1]:
import pyodbc
import pandas as pd
import warnings

from sqlalchemy import create_engine
import urllib

warnings.filterwarnings('ignore', category=UserWarning)

Criando conexão

In [ ]:
import os
senha = os.getenv('senha_bd_simples')

conn = pyodbc.connect(
    "DRIVER={ODBC Driver 17 for SQL Server};" \
    "SERVER=den1.mssql7.gear.host;" \
    "DATABASE=vendasbicyclon;" \
    "UID=vendasbicyclon;" \
    f"PWD={senha};" \
    "MARS_Connection=Yes"
)

In [10]:
def exe_query(query):
    return pd.read_sql(query, conn)

lista_resposta = []
def exe_varias_querys(*args):
    for consulta in args:
        lista_resposta.append(pd.read_sql(consulta, conn))
    return lista_resposta

Criando conexão com SQLAlchemy

In [13]:
params = urllib.parse.quote_plus(
    "DRIVER={ODBC Driver 17 for SQL Server};" \
    "SERVER=den1.mssql7.gear.host;" \
    "DATABASE=vendasbicyclon;" \
    "UID=vendasbicyclon;" \
    "PWD=senha@clarify;" \
    "MARS_Connection=Yes"
)

engine = create_engine(f'mssql+pyodbc:///?odbc_connect={params}')

Produtos, total de pedidos e qtd vendida

In [ ]:
q = """ 
SELECT 
    produtos.nomeProduto,
    count(itensPedido.idItem) as total_pedidos,
    sum(itensPedido.quantidade) as qtd_vendido

    from produtos join itensPedido 
        on produtos.idProduto = itensPedido.fkProduto
        group by produtos.nomeProduto
"""

exe_query(q)

Consolidado - VIEW

In [ ]:
q_vw = """ 
if OBJECT_ID('vwFatoVendas', 'V') is not null 
    drop view vwFatoVendas

exec('
    create view vwFatoVendas as 
    select
        pedidos.idPedido,
        pedidos.dataPedido,
        clientes.nomeCliente,
        vendedores.nomeVendedor,
        regionais.nomeRegional,
        produtos.nomeProduto,
        linhasProdutos.nomeLinhaProduto,
        itensPedido.quantidade,
        itensPedido.precoUnitario,
        itensPedido.desconto,
        (itensPedido.quantidade * (
            itensPedido.precoUnitario - itensPedido.desconto)
            ) as valor_total

        from pedidos
        join clientes  on pedidos.fkCliente = clientes.idCliente
        join vendedores on pedidos.fkVendedor = vendedores.idVendedor
        join regionais on  vendedores.fkRegional = regionais.idRegional
        join itensPedido on itensPedido.fkPedido = pedidos.idPedido
        join produtos on  itensPedido.fkProduto =  produtos.idProduto
        join linhasProdutos on produtos.fkLinhaProduto = linhasProdutos.idLinhaProduto
')
"""

cursor = conn.cursor()
cursor.execute(q_vw)
conn.commit()

df_fatovendas = pd.read_sql('select * from vwFatoVendas', conn)

df_fatovendas.columns

In [1]:
import os

print(os.getenv('senha_bd_simples'))

senha@clarify
